In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import scipy.sparse as sp
from scipy.special import expit
import pickle
import csv
import scipy.sparse as sparse
import copy
import itertools
from lightfm import LightFM
import lightfm.evaluation
import sys
import implicit
plt.style.use('ggplot')

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
# raw_df = pd.read_csv('cat_de_3year.csv', encoding = "utf-8")#, nrows = 100000)
raw_df = pd.read_csv('enreached_data_cat_de.csv', encoding = "utf-8")#, nrows = 100000)

In [3]:
raw_df.shape

(8510500, 21)

In [4]:
raw_df['CUS_N_KEY_CUSTOMER'].unique().size

799943

In [5]:
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
0,209091111,49807309,0,16-FEB-15,16715,15665,6412112,1.0,4220,4097,...,3481.0,3481,3481.0 Dog Design Decke Hundeschar - 60x80 cm,3481.0 Dog Design Decke Hundeschar - 60x80 cm,Hundebetten & Hundekörbe,Hundedecken,Hundedecken,Hundedecken,Dog Design,Canned Food Cat / Dog
1,203263548,48465719,0,12-JAN-15,1323999,11459,3116936,1.0,26073,19403,...,13468.0,13468,13468.0 Royal Canin Fit Adult Cat - 400 g,13468.0 Royal Canin Fit Adult Cat - 400 g,Dry Cat Food,Royal Canin Feline,Royal Canin Cat Health Nutrition,Royal Canin Cat Health Nutrition,Royal Canin,snacks
2,203263549,48465719,0,12-JAN-15,1324000,11460,3116936,1.0,26072,19403,...,13467.1,13467,13467.1 Happy Dog Supreme Sensible New Zealand...,13467.1 Happy Dog Supreme Sensible New Zealand...,Dry Dog Food,Happy Dog Supreme,Happy Dog Supreme Sensible,Happy Dog Supreme Sensible,Interquell,snacks
3,226948694,53944802,0,06-JUN-15,131,16891,7438119,1.0,9499,1164,...,6107.4,6107,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,Filterzubehör / -ersatz,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Eheim,"small cat / dog acc., container and local merc..."
4,207176125,49355253,0,04-FEB-15,560905,6457239,4012659,1.0,30297,21412,...,15067.0,15067,"15067.0 Forza10 Maintenance Probierpaket, 6 x ...","15067.0 Forza10 Maintenance Probierpaket, 6 x ...",Katzenfutter nass,Forza10,Forza10,Forza10,Forza10,Dry Food Cat / Dog


In [6]:
## to start let us take a sample of our data
sample_df = raw_df # .sample(10000000)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
0,209091111,49807309,0,16-FEB-15,16715,15665,6412112,1.0,4220,4097,...,3481.0,3481,3481.0 Dog Design Decke Hundeschar - 60x80 cm,3481.0 Dog Design Decke Hundeschar - 60x80 cm,Hundebetten & Hundekörbe,Hundedecken,Hundedecken,Hundedecken,Dog Design,Canned Food Cat / Dog
1,203263548,48465719,0,12-JAN-15,1323999,11459,3116936,1.0,26073,19403,...,13468.0,13468,13468.0 Royal Canin Fit Adult Cat - 400 g,13468.0 Royal Canin Fit Adult Cat - 400 g,Dry Cat Food,Royal Canin Feline,Royal Canin Cat Health Nutrition,Royal Canin Cat Health Nutrition,Royal Canin,snacks
2,203263549,48465719,0,12-JAN-15,1324000,11460,3116936,1.0,26072,19403,...,13467.1,13467,13467.1 Happy Dog Supreme Sensible New Zealand...,13467.1 Happy Dog Supreme Sensible New Zealand...,Dry Dog Food,Happy Dog Supreme,Happy Dog Supreme Sensible,Happy Dog Supreme Sensible,Interquell,snacks
3,226948694,53944802,0,06-JUN-15,131,16891,7438119,1.0,9499,1164,...,6107.4,6107,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,Filterzubehör / -ersatz,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Eheim,"small cat / dog acc., container and local merc..."
4,207176125,49355253,0,04-FEB-15,560905,6457239,4012659,1.0,30297,21412,...,15067.0,15067,"15067.0 Forza10 Maintenance Probierpaket, 6 x ...","15067.0 Forza10 Maintenance Probierpaket, 6 x ...",Katzenfutter nass,Forza10,Forza10,Forza10,Forza10,Dry Food Cat / Dog


In [7]:
# remove non-used columns
#sample_df.drop(['SCM_N_ORDER_POS_NR','SCM_N_ORDER_NR_NC', 'SCM_N_ORDER_NR_NC', 'PPR_N_KEY_PHYS_PRODUCT', 'SCM_N_ORDER_NR', 'ART_N_KEY_ARTICLE','PPR_N_PHY_PRODUCT_ID'], axis=1, inplace=True)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
0,209091111,49807309,0,16-FEB-15,16715,15665,6412112,1.0,4220,4097,...,3481.0,3481,3481.0 Dog Design Decke Hundeschar - 60x80 cm,3481.0 Dog Design Decke Hundeschar - 60x80 cm,Hundebetten & Hundekörbe,Hundedecken,Hundedecken,Hundedecken,Dog Design,Canned Food Cat / Dog
1,203263548,48465719,0,12-JAN-15,1323999,11459,3116936,1.0,26073,19403,...,13468.0,13468,13468.0 Royal Canin Fit Adult Cat - 400 g,13468.0 Royal Canin Fit Adult Cat - 400 g,Dry Cat Food,Royal Canin Feline,Royal Canin Cat Health Nutrition,Royal Canin Cat Health Nutrition,Royal Canin,snacks
2,203263549,48465719,0,12-JAN-15,1324000,11460,3116936,1.0,26072,19403,...,13467.1,13467,13467.1 Happy Dog Supreme Sensible New Zealand...,13467.1 Happy Dog Supreme Sensible New Zealand...,Dry Dog Food,Happy Dog Supreme,Happy Dog Supreme Sensible,Happy Dog Supreme Sensible,Interquell,snacks
3,226948694,53944802,0,06-JUN-15,131,16891,7438119,1.0,9499,1164,...,6107.4,6107,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,Filterzubehör / -ersatz,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Eheim,"small cat / dog acc., container and local merc..."
4,207176125,49355253,0,04-FEB-15,560905,6457239,4012659,1.0,30297,21412,...,15067.0,15067,"15067.0 Forza10 Maintenance Probierpaket, 6 x ...","15067.0 Forza10 Maintenance Probierpaket, 6 x ...",Katzenfutter nass,Forza10,Forza10,Forza10,Forza10,Dry Food Cat / Dog


In [8]:
sample_renamed_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'uid', 
                                                   'ART_N_KEY_ARTICLE': 'mid', 
                                                   'TDT_T_KEY_ORDER_DATE': 'date', # it looks like this works a bit better
                                                   'SCM_D_QUANTITY': 'eventStrength'})
sample_renamed_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,date,mid,PPR_N_KEY_PHYS_PRODUCT,uid,eventStrength,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
0,209091111,49807309,0,16-FEB-15,16715,15665,6412112,1.0,4220,4097,...,3481.0,3481,3481.0 Dog Design Decke Hundeschar - 60x80 cm,3481.0 Dog Design Decke Hundeschar - 60x80 cm,Hundebetten & Hundekörbe,Hundedecken,Hundedecken,Hundedecken,Dog Design,Canned Food Cat / Dog
1,203263548,48465719,0,12-JAN-15,1323999,11459,3116936,1.0,26073,19403,...,13468.0,13468,13468.0 Royal Canin Fit Adult Cat - 400 g,13468.0 Royal Canin Fit Adult Cat - 400 g,Dry Cat Food,Royal Canin Feline,Royal Canin Cat Health Nutrition,Royal Canin Cat Health Nutrition,Royal Canin,snacks
2,203263549,48465719,0,12-JAN-15,1324000,11460,3116936,1.0,26072,19403,...,13467.1,13467,13467.1 Happy Dog Supreme Sensible New Zealand...,13467.1 Happy Dog Supreme Sensible New Zealand...,Dry Dog Food,Happy Dog Supreme,Happy Dog Supreme Sensible,Happy Dog Supreme Sensible,Interquell,snacks
3,226948694,53944802,0,06-JUN-15,131,16891,7438119,1.0,9499,1164,...,6107.4,6107,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,6107.4 EHEIM Schlauchklemme - zu Ø 25 / 34 mm,Filterzubehör / -ersatz,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Schlauch-/Rohrzubehör,Eheim,"small cat / dog acc., container and local merc..."
4,207176125,49355253,0,04-FEB-15,560905,6457239,4012659,1.0,30297,21412,...,15067.0,15067,"15067.0 Forza10 Maintenance Probierpaket, 6 x ...","15067.0 Forza10 Maintenance Probierpaket, 6 x ...",Katzenfutter nass,Forza10,Forza10,Forza10,Forza10,Dry Food Cat / Dog


In [9]:
n_users = sample_renamed_df.uid.unique().shape[0]
n_items = sample_renamed_df.mid.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(sample_renamed_df.shape[0]) / float(n_users*n_items) * 100))

Number of users: 799943
Number of models: 3491
Sparsity: 0.305%


In [10]:
sample_renamed_df.uid[2]

3116936

In [11]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 15783502].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
5427842,338243366,79528354,79528354,19-DEC-16,1916791,11235,15783502,2.3965,11078,9538,...,6736.0,6736,6736.0 Landschildkröten -,6736.0 Landschildkröten -,Tote Artikel Rep,Zoo Med,Zoo Med,Zoo Med,Falken,"small cat / dog acc., container and local merc..."
5427843,338243364,79528354,79528354,19-DEC-16,1545998,12085,15783502,0.3026,26186,19451,...,13496.3,13496,13496.3 Pro Plan Adult Medium Sensitive Skin O...,13496.3 Pro Plan Adult Medium Sensitive Skin O...,Dry Dog Food,Purina Pro Plan,Pro Plan Adult,Pro Plan Adult,Nestle,"small cat / dog acc., container and local merc..."
5427859,338243364,79528354,79528354,19-DEC-16,1545998,12090,15783502,0.1831,26184,19449,...,13496.1,13496,13496.1 Pro Plan Adult Medium Sensitive Skin O...,13496.1 Pro Plan Adult Medium Sensitive Skin O...,Dry Dog Food,Purina Pro Plan,Pro Plan Adult,Pro Plan Adult,Nestle,"small cat / dog acc., container and local merc..."
6691043,392411314,91906904,0,03-AUG-17,534252,4555549,15783502,1.0000,29320,20787,...,14629.0,14629,14629.0 Hunter Norwegian Harness - Size S: max...,14629.0 Hunter Norwegian Harness - Size S: max...,Dog Leads & Dog Collars,Dog Harness,Hunter Harnesses,Hunter Harnesses,Hunter,snacks
7126556,401149584,94000372,0,08-SEP-17,416175,11332,15783502,1.0000,26920,19856,...,13729.2,13729,13729.2 2 kg Gimpet Trockenfutter + 100 g Gimp...,13729.2 2 kg Gimpet Trockenfutter + 100 g Gimp...,Katzenfutter trocken,Gimpet,Gimpet,Gimpet,Gimborn Spezialitaeten,branded acc. cat / dog
8096068,445970420,104566072,0,13-FEB-18,423994,11331,15783502,1.0000,27175,19856,...,13836.1,13836,"13836.1 Cat Exclusive 156 g, 10 + 2 gratis - ...","13836.1 Cat Exclusive 156 g, 10 + 2 gratis - ...",Katzenfutter nass,Cat Exclusive,Cat Exclusive,Cat Exclusive,Iwersen,branded acc. cat / dog


In [12]:
raw_df['ART_N_KEY_ARTICLE'].unique().size

3491

In [13]:
raw_df['PPR_N_PHY_ARTICLE_ID'].unique().size

659

In [14]:
raw_df['PPR_N_PHY_PRODUCT_ID'].unique().size

352

In [15]:
raw_df[raw_df['ART_N_KEY_ARTICLE'] == 1281816].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC


While implicit recommendations excel where data is sparse, it can often be helpful to make the interactions matrix a little more dense. We limited our data collection to models that had at least 5 likes. However, it may not be the case that every user has liked at least 5 models. Let's go ahead and knock out users that liked fewer than 5 models. This could possibly mean that some models end up with fewer than 5 likes once these users are knocked out, so we will have to alternate back and forth knocking users and models out until things stabilize.

In [16]:
def threshold_likes(df, uid_min, mid_min):
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby('uid').mid.count()
        df = df[~df.uid.isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby('mid').uid.count()
        df = df[~df.mid.isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('uid').mid.count().min() >= mid_min)
    assert(df.groupby('mid').uid.count().min() >= uid_min)
    
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [17]:
df_lim = threshold_likes(sample_renamed_df, 5, 5)

Starting likes info
Number of users: 799943
Number of models: 3491
Sparsity: 0.305%
Ending likes info
Number of users: 324078
Number of models: 3365
Sparsity: 0.701%


Nice, we're above 0.1% which should be suitable for making decent recommendations. We now need to map each uid and mid to a row and column, respectively, for our interactions, or "likes" matrix. This can be done simply with Python dictionaries

In [18]:
# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df_lim.mid.unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_lim.uid.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

The last step is to actually build the matrix. We will use sparse matrices so as not to take up too much memory. Sparse matrices are tricky because they come in many forms, and there are giant performance tradeoffs between them. Below is a super slow way to build a likes matrix. I tried running %%timeit but got bored waiting for it to finish.

In [19]:
def map_ids(row, mapper):
    return mapper[row]

In [20]:
%%timeit
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

10.5 s ± 212 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

## Cross-validation: Splitsville

Okay, we got a likes matrix and need to split it into training and test matrices. I do this a bit trickily (which is maybe a word?). I would like to track precision@k as my optimization metric later. A k of 5 would be nice. However, if I move 5 items from training to test for some of the users, then they may not have any data left in the training set (remember they had a minimum 5 likes). Thus, the train_test_split only looks for people who have at least 2*k (10 in this case) likes before moving some of their data to the test set. This obviously biases our cross-validation towards users with more likes. So it goe

In [22]:
def train_test_split(ratings, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = np.random.choice(ratings.getrow(user).indices, 
                                        size=split_count, 
                                        replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [23]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

## Cross-validation: Grid search

Now that the data is split into training and test matrices, let's run a giant grid search to optimize our hyperparameters. We have four parameters that we would like to optimize:
1. num_factors: The number of latent factors, or degree of dimensionality in our model.
2. regularization: Scale of regularization for both user and item factors.
3. alpha: Our confidence scaling term.
4. iterations: Number of iterations to run Alternating Least Squares optimization.

I am going to track mean squared error (MSE) and precision at k (p@k), but I mainly care about the later. I've written some functions below to help with metric calculations and making the training log printout look nice. I am going to calculate a bunch of learning curves (that is, evaluate performance metrics at various stages of the training process) for a bunch of different hyperparameter combinations. Props to scikit-learn for being open source and letting me basically crib their GridSearchCV code

In [24]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, ratings, user_index=None):
    preds = model.predict_for_customers()
    if user_index:
        return mean_squared_error(ratings[user_index, :].toarray().ravel(),
                                  preds[user_index, :].ravel())
    
    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

In [25]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
    predictions = model.predict_for_customers()
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        top_k = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)        

In [26]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [27]:
def learning_curve(model, train, test, epochs, k=5, user_index=None):
    if not user_index:
        user_index = range(train.shape[0])
    prev_epoch = 0
    train_precision = []
    train_mse = []
    test_precision = []
    test_mse = []
    
    headers = ['epochs', 'p@k train', 'p@k test',
               'mse train', 'mse test']
    print_log(headers, header=True)
    
    for epoch in epochs:
        model.iterations = epoch - prev_epoch
        if not hasattr(model, 'user_vectors'):
            model.fit(train)
        else:
            model.fit_partial(train)
        train_mse.append(calculate_mse(model, train, user_index))
        train_precision.append(precision_at_k(model, train, k, user_index))
        test_mse.append(calculate_mse(model, test, user_index))
        test_precision.append(precision_at_k(model, test, k, user_index))
        row = [epoch, train_precision[-1], test_precision[-1],
               train_mse[-1], test_mse[-1]]
        print_log(row)
        prev_epoch = epoch
    return model, train_precision, train_mse, test_precision, test_mse

In [28]:
def grid_search_learning_curve(base_model, train, test, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, train_mse, test_patk, test_mse = learning_curve(this_model, train, test,
                                                                epochs, k=patk, user_index=user_index)
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk},
                       'mse': {'train': train_mse, 'test': test_mse}})
    return curves

Please note that the below parameter grid is fucking huge and took like 2 days to run on my 6-year old 4-core i5. It turns out that the performance metrics functions are actually a good bit slower than the training process. These functions could be simply paralellized - something for me to do on a later date.

In [29]:
# param_grid = {'num_factors': [10, 20, 40, 80, 120],
#              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
#              'alpha': [1, 10, 50, 100, 500, 1000]}
param_grid = {'num_factors': [10, 20],
              'regularization': [0.0, 1e-5],
              'alpha': [1 ]}

In [30]:
#base_model = implicit.als.AlternatingLeastSquares(factors=10)

In [31]:
#curves = grid_search_learning_curve(base_model, train, test,
#                                    param_grid,
#                                    user_index=user_index,
#                                    patk=5)

# fixing implicit implementation:

In [32]:
base_model = implicit.als.AlternatingLeastSquares()

In [33]:
likes_t = likes.transpose()
likes_t.shape

(3365, 324078)

In [34]:
# train the model on a sparse matrix of item/user/confidence weights
base_model.fit(likes_t)

100%|██████████| 15.0/15 [00:22<00:00,  1.67s/it]


In [35]:
user_items = likes_t.T.tocsr()

In [36]:
user_items.shape

(324078, 3365)

In [37]:
recommendations = base_model.recommend(uid_to_idx[8790000], user_items)

In [126]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 8790000].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
32301,254832173,60433469,0,14-NOV-15,1481653,16329,8790000,0.3328,14618,11809,...,9282.3,9282,9282.3 Megapack Kitekat in Soße 24 x 415 g - M...,9282.3 Megapack Kitekat in Soße 24 x 415 g - M...,Katzenfutter nass,Kitekat,Kitekat,Kitekat,Mars Petcare (Retail),Canned Food Cat / Dog
32302,254832173,60433469,0,14-NOV-15,1481653,16320,8790000,0.3329,26414,11809,...,13572.2,13572,13572.2 Rubin Cat Tree - Blue,13572.2 Rubin Cat Tree - Blue,Cat Trees & Cat Scratching Posts,Medium Cat Trees,Medium Cat Trees,Medium Cat Trees,Trixie,Canned Food Cat / Dog
32303,254832173,60433469,0,14-NOV-15,1481653,16328,8790000,0.3310,14619,11809,...,9282.4,9282,9282.4 Megapack Kitekat in Soße 24 x 415 g - Rind,9282.4 Megapack Kitekat in Soße 24 x 415 g - Rind,Katzenfutter nass,Kitekat,Kitekat,Kitekat,Mars Petcare (Retail),Canned Food Cat / Dog
32313,254832163,60433469,0,14-NOV-15,1394768,917505,8790000,1.0000,27902,7785,...,14073.1,14073,"14073.1 Dog Bed Arnold - 70 x 100 cm, light bl...","14073.1 Dog Bed Arnold - 70 x 100 cm, light bl...",Dog Beds & Dog Baskets,Dog Dens,Square Dog Beds,Square Dog Beds,InTierior,Canned Food Cat / Dog
32314,254832167,60433469,0,14-NOV-15,1086182,5049882,8790000,1.0000,29463,20825,...,14689.1,14689,"14689.1 Plush Paws Dog Bed, cream - Ø 80 cm","14689.1 Plush Paws Dog Bed, cream - Ø 80 cm",Dog Beds & Dog Baskets,Dog Dens,Round Dog Beds,Round Dog Beds,Trixie,Canned Food Cat / Dog
32325,254832168,60433469,0,14-NOV-15,927412,11459,8790000,1.0000,26073,19403,...,13468.0,13468,13468.0 Royal Canin Fit Adult Cat - 400 g,13468.0 Royal Canin Fit Adult Cat - 400 g,Dry Cat Food,Royal Canin Feline,Royal Canin Cat Health Nutrition,Royal Canin Cat Health Nutrition,Royal Canin,snacks
32326,254832173,60433469,0,14-NOV-15,1481653,16326,8790000,0.3329,14616,11809,...,9282.1,9282,9282.1 Megapack Kitekat in Soße 24 x 415 g - z...,9282.1 Megapack Kitekat in Soße 24 x 415 g - z...,Katzenfutter nass,Kitekat,Kitekat,Kitekat,Mars Petcare (Retail),Canned Food Cat / Dog
32327,254832173,60433469,0,14-NOV-15,1481653,16323,8790000,0.3338,14617,11809,...,9282.2,9282,9282.2 Megapack Kitekat in Soße 24 x 415 g - W...,9282.2 Megapack Kitekat in Soße 24 x 415 g - W...,Katzenfutter nass,Kitekat,Kitekat,Kitekat,Mars Petcare (Retail),Canned Food Cat / Dog
32328,254832173,60433469,0,14-NOV-15,1481653,16327,8790000,0.3366,14620,11809,...,9282.5,9282,9282.5 Megapack Kitekat in Soße 24 x 415 g - T...,9282.5 Megapack Kitekat in Soße 24 x 415 g - T...,Katzenfutter nass,Kitekat,Kitekat,Kitekat,Mars Petcare (Retail),Canned Food Cat / Dog
32333,254832174,60433469,0,14-NOV-15,15402,13084,8790000,1.0000,15834,9595,...,9748.1,9748,9748.1 Elch-Anhänger,9748.1 Elch-Anhänger,Bozita,Bozita Dog,Bozita Dog,Bozita Dog,Doggy-Bozita,"Premium Brands by P, Nestle and Hills"


In [38]:
recommendations

[(1020, 1.1923853),
 (647, 1.1649411),
 (116, 1.0470644),
 (469, 0.98783112),
 (783, 0.95302349),
 (774, 0.94534945),
 (740, 0.94192976),
 (1001, 0.9388625),
 (1090, 0.88643193),
 (1441, 0.88034236)]

In [111]:
raw_df[raw_df['ART_N_KEY_ARTICLE'] == idx_to_mid[1020]]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC
4158,211006475,50251451,0,28-FEB-15,1394671,16108,10858798,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
6043,210218251,50072658,0,23-FEB-15,1394671,16108,8481108,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
8601,216735425,51565292,0,03-APR-15,1394671,16108,10340836,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
10570,220106578,52356258,0,25-APR-15,1394671,16108,10520083,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
11857,212070244,50493484,0,05-MAR-15,1394671,16108,3967432,5.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
14893,316001764,74496574,0,15-SEP-16,1394671,16108,1844272,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
15773,316374056,74581272,0,17-SEP-16,1394671,16108,4043940,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
16193,206850720,49279013,0,02-FEB-15,1394671,16108,454110,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
23804,215820617,51369114,0,29-MAR-15,1394671,16108,6955147,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog
26382,247485081,58713539,0,05-OCT-15,1394671,16108,2559980,1.0,18603,10178,...,10677.3,10677,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,10677.3 PetSafe Spray Anti-Bell-Halsband - Nac...,Hundesport & Erziehung,Erziehungshalsband & Antibell,Antibell-Halsband,Antibell-Halsband,radio systems,Canned Food Cat / Dog


In [110]:
raw_df[raw_df['ART_C_PROD_NUMBER'] == 7016]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_ART_NUMBER,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC


In [93]:
# find related items
# related = base_model.similar_items(mid_to_idx[])
related = base_model.similar_items(116)
related

[(116, 1.0000001),
 (1118, 0.97208744),
 (752, 0.94232947),
 (115, 0.92849022),
 (1180, 0.92254758),
 (1205, 0.90651268),
 (1242, 0.90290916),
 (1241, 0.89986324),
 (1208, 0.87184554),
 (1001, 0.70292145)]

In [124]:
for aid , _ in recommendations:
    tmp = raw_df[raw_df['ART_N_KEY_ARTICLE'] == idx_to_mid[aid]]['ART_C_PROD_NUMBER'].unique()
    print (tmp)
    print (raw_df[raw_df['ART_C_PROD_NUMBER'] == tmp[0]]['ART_V_ART_DESCRIPTION_1'].iloc[0])

[10677]
10677.3 PetSafe Spray Anti-Bell-Halsband - Nachfüllpatrone 70 g
[9357]
9357.0 Bozita Robur Breeder & Puppy 30/15 - 15 kg
[10039]
10039.17 Sparpaket Miamor Pastete 12 x 85 g - Shrimps
[9357]
9357.0 Bozita Robur Breeder & Puppy 30/15 - 15 kg
[7224]
7224.0 Pro Plan Adult Sensitive Lachs & Reis Hundefutter - 800 g
[7221]
7221.0 Eco Food Krokant - 4 kg
[5847]
5847.3 Kunststoff Putzbox - schwarz
[11031]
11031.0 GrünerTraum Mix - 500 g
[6377]
6377.1 Nutro Max Cat Nassfutter Multipack - Huhn und Lamm, 5 x 156 g
[9759]
9759.4 Fluval Schaumstoffpatronen - Multipack 3 x 2 Schaumstoffpatronen für 204/205/206/304/305/306


## Let us load the images

In [130]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [129]:
img_df = pd.read_csv('ps_id_to_img.csv', encoding = "utf-8")#, nrows = 100000)
img_df.head()

,IMG_NAME,PS_ID,PS_B_ID,PS_SHOP_IDENTIFIER
0,tischset_b_1.jpg,6116,12165,7905
1,7390_1.jpg,5603,12173,7390
2,friskiesduofresh7232_1.jpg,5447,9919,7232
3,eukanubatestthebest_1.jpg,5886,10678,7673
4,7322_1.jpg,5536,10714,7322


In [142]:
def name_to_url(name):
    ## select 'https://media.zooplus.com/bilder/'||SUBSTR (b_pfad, instr(b_pfad,'.',-1,1)-1, 1)||'/140/'||B_PFAD from mv_prod_datafeed_export;
    return 'https://media.zooplus.com/bilder/' + name[name.find('.')-1] + '/140/'+ name

name_to_url_lambda = lambda x: 'https://media.zooplus.com/bilder/' + x[x.find('.')-1] + '/140/'+ x

img_df['img_url'] = img_df['IMG_NAME'].apply(name_to_url_lambda)

In [164]:
for aid , _ in recommendations:
    #print(idx_to_mid[aid])
    #ART_C_PROD_NUMBER
    ps_id = raw_df[raw_df['ART_N_KEY_ARTICLE'] == idx_to_mid[aid]]['ART_C_PROD_NUMBER'].unique()[0]
    # print(ps_id)
    img_url = img_df[img_df['PS_ID'] == ps_id]['img_url'].iloc[0]
    print(img_url)

https://media.zooplus.com/bilder/1/140/1376_Trixie_Fuenfeck_1.jpg
https://media.zooplus.com/bilder/1/140/10143_new_1.jpg
https://media.zooplus.com/bilder/2/140/13019_2.jpg
https://media.zooplus.com/bilder/1/140/10143_new_1.jpg
https://media.zooplus.com/bilder/1/140/10162_1.jpg
https://media.zooplus.com/bilder/1/140/kette_1.jpg
https://media.zooplus.com/bilder/1/140/koziolelch_1.jpg
https://media.zooplus.com/bilder/1/140/77227_pla_bozita_pate_lamm_625g_1.jpg
https://media.zooplus.com/bilder/1/140/9309_1.jpg
https://media.zooplus.com/bilder/1/140/12732_1.jpg


In [ ]:
from IPython.display import display, HTML

def display_thumbs(thumbs, N=5):
    thumb_html = "<img style='width: 160px; margin: 0px; \
                  float: left; border: 1px solid black;' src='{}' />"
    images = ''
    display(HTML('<font size=5>'+'Input Model'+'</font>'))
    display(HTML(thumb_html.format(thumbs[0])))
    display(HTML('<font size=5>'+'Similar Models'+'</font>'))

    for url in thumbs[1:N+1]:
        images += thumb_html.format(url)
    display(HTML(images))